# Sentinel-2 Composite Generation

Generate a image composite with Google Earth Engine and Cloud Score+

In [1]:
import ee
import geemap
import geopandas
from deepfreezer.utils import ROOT_DIR

from typing import Literal

In [2]:
ee.Initialize(project="lqg-global-glaciers")

In [3]:
def create_composite_cs(
    image_col: ee.ImageCollection,
    qa_band: Literal["cs", "cs_cdf"] = "cs_cdf",
    threshold: float = 0.6,
) -> ee.Image:
    """Create a Sentinel-2 composite image using the Google Cloud Score+"""
    csp = ee.ImageCollection("GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED")

    return (
        image_col.linkCollection(csp, [qa_band])
        .map(lambda img: img.updateMask(img.select(qa_band).gte(threshold)))
        .median()
    )


In [4]:
def mask_clouds_qa(image: ee.Image) -> ee.Image:
    """Masks clouds in a Sentinel-2 image using the QA band.

    Args:
        image (ee.Image): A Sentinel-2 image.

    Returns:
        ee.Image: A cloud-masked Sentinel-2 image.
    """
    qa = image.select("QA60")

    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloud_bit_mask = 1 << 10
    cirrus_bit_mask = 1 << 11

    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloud_bit_mask).eq(0).And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))

    return image.updateMask(mask)


def create_composite_qa(image_col: ee.ImageCollection) -> ee.Image:
    # Pre-filter to get less cloudy granules.
    return (
        image_col.filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 20))
        .map(mask_clouds_qa)
        .median()
    )

In [5]:
s2 = ee.ImageCollection("COPERNICUS/S2_HARMONIZED")

QA_BAND = "cs_cdf"
CLEAR_THRESHOLD = 0.5

In [6]:
# Import SGI
sgi = geopandas.read_file(
    ROOT_DIR / "data/raw/inventory_sgi2016_r2020/SGI_2016_glaciers.shp"
)

# Clean file
sgi = sgi.drop(columns="pk_glacier")
sgi.gid = sgi.gid.astype(int)
sgi = sgi.set_index("gid", verify_integrity=True).sort_index().to_crs("EPSG:4326")
sgi_bounds = sgi.total_bounds
sgi_bounds

array([ 6.82208749, 45.86092842, 10.41511278, 47.25128772])

In [7]:
# Convert GeoDataFrame to FeatureCollection
fc_sgi = geemap.geopandas_to_ee(sgi)
fc_sgi

/Users/lgentn/Repositories/glaciermap/.venv/lib/python3.13/site-packages/IPython/core/formatters.py:406: UserWarning: Getting info failed with: 'Request payload size exceeds the limit: 10485760 bytes.'. Falling back to string repr.
  return method()


In [8]:
roi = ee.Geometry.Rectangle(*sgi_bounds)

In [9]:
s2_filtered = s2.filterDate("2023-05-01", "2023-07-31").filterBounds(roi)
composite_cs = create_composite_cs(
    s2_filtered,
    qa_band=QA_BAND,
    threshold=CLEAR_THRESHOLD,
)
composite_qa = create_composite_qa(s2_filtered)

visualization = {
    "min": 0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],
}

m = geemap.Map()
m.center_object(roi, 12)
m.add_layer(composite_cs, visualization, "Cloud Score+")
m.add_layer(composite_qa, visualization, "QA60")
m

Map(center=[46.567199594409665, 8.618600132009618], controls=(WidgetControl(options=['position', 'transparent_…